<a href="https://colab.research.google.com/github/kovac04/nets/blob/main/karpathy_tutorials/GPT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy
import torch
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt

In [3]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-08-23 22:43:47--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.06s   

2024-08-23 22:43:47 (17.7 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [4]:
with open("input.txt" , 'r', encoding='utf-8') as f:
  text = f.read()

In [5]:
len(text), text[:1000]

(1115394,
 "First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you know Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us kill him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be done: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citizens, the patricians good.\nWhat authority surfeits on would relieve us: if they\nwould yield us but the superfluity, while it were\nwholesome, we might guess they relieved us humanely;\nbut they think we are too dear: the leanness that\nafflicts us, the object of our misery, is as an\ninventory to particularise their abundance; our\nsufferance is a gain to them Let us revenge this with\nour pikes, ere we become rakes: for the gods know I\nspeak this

In [6]:
all_chars = sorted(list(set(text)))
all_chars = ''.join(all_chars)
vocab_size = len(all_chars)
vocab_size, all_chars

(65, "\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz")

In [7]:
# "tokenizer"
stoi = {s:i for i,s in enumerate(all_chars)}
itos = {i:s for i,s in enumerate(all_chars)}
encode = lambda x: [stoi[i] for i in x]
decode = lambda x: ''.join(itos[i] for i in x)

In [8]:
data = torch.tensor(encode(text) , dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [10]:
# want 4 indices on which we will start our block_size long sequences, pluck out block_size chunks from text, turn them into arrays each
torch.manual_seed(1337)
batch_size = 4
block_size = 8
def get_batch(split):
  data = train_data if split=='train' else val_data
  ix = torch.randint(0, len(data)-block_size, (batch_size,))
  x = torch.stack([data[index:index+block_size] for index in ix])
  y = torch.stack([data[index+1:index+block_size+1] for index in ix])
  return x,y

In [11]:
xb,yb = get_batch('train')

In [12]:
for i in range(batch_size):
  for j in range(block_size):
    context = xb[i,:j+1]
    label = yb[i,j]
    print(f"{context.tolist()} -> {label}")

[24] -> 43
[24, 43] -> 58
[24, 43, 58] -> 5
[24, 43, 58, 5] -> 57
[24, 43, 58, 5, 57] -> 1
[24, 43, 58, 5, 57, 1] -> 46
[24, 43, 58, 5, 57, 1, 46] -> 43
[24, 43, 58, 5, 57, 1, 46, 43] -> 39
[44] -> 53
[44, 53] -> 56
[44, 53, 56] -> 1
[44, 53, 56, 1] -> 58
[44, 53, 56, 1, 58] -> 46
[44, 53, 56, 1, 58, 46] -> 39
[44, 53, 56, 1, 58, 46, 39] -> 58
[44, 53, 56, 1, 58, 46, 39, 58] -> 1
[52] -> 58
[52, 58] -> 1
[52, 58, 1] -> 58
[52, 58, 1, 58] -> 46
[52, 58, 1, 58, 46] -> 39
[52, 58, 1, 58, 46, 39] -> 58
[52, 58, 1, 58, 46, 39, 58] -> 1
[52, 58, 1, 58, 46, 39, 58, 1] -> 46
[25] -> 17
[25, 17] -> 27
[25, 17, 27] -> 10
[25, 17, 27, 10] -> 0
[25, 17, 27, 10, 0] -> 21
[25, 17, 27, 10, 0, 21] -> 1
[25, 17, 27, 10, 0, 21, 1] -> 54
[25, 17, 27, 10, 0, 21, 1, 54] -> 39


In [33]:
class BigramLanguageModel(nn.Module):
  def __init__(self, vocab_size):
    super().__init__()                                            # same table we had in makemorer "P". Each row's probs ∑ = 1
    self.token_embedding_table = nn.Embedding(vocab_size, vocab_size) # row index -> what char we are looking at (input)
                                                                      # column index -> highest prob next char (output)
  def forward(self, x, targets=None):
    logits = self.token_embedding_table(x) # returns Batch(4), Time(8), Channel(65). For each input in all 4 batches
                                           # get a distribution over the next char
    B, T, C = logits.shape
    logits = logits.view(B*T, C)
    if targets is not None:
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets) # CE expects C as 2nd dimension so have to reshape logits and effectively merge all chars into 1 batch (get rid of batch dim)
      return logits, loss
    else:
      return logits

  def generate(self, x, max_new_tokens):
    # x is (B,T) = current contexts (4,8) for each batch. Want to generate max_new_tokens more chars for each batch
    for _ in range(max_new_tokens):
      logits, _ = self(x)
      logits = logits[:,-1:] # take only the last character's probability distribution for next char, for each batch. B,T,C -> B,C
      probs = F.softmax(logits, dim = 1) # B,C
      idx_next = torch.multinomial(probs, num_samples = 1) # B,C -> B,1
      idx = torch.cat((idx,idx_next), dim =1) # B,T+1   idx becomes a sequence of new chars that index into
    return idx

In [38]:
model = BigramLanguageModel(vocab_size)
logits, loss = model(xb,yb)
print(logits.shape, loss)
#print(torch.argmax(logits[0,0]))
#print(logits[0,0,42])
#print(xb[0,0])
print(yb.shape)

torch.Size([32, 65]) tensor(4.6893, grad_fn=<NllLossBackward0>)
torch.Size([4, 8])
